In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests

import pathlib

import time

from email.message import EmailMessage
import ssl
import smtplib
import getpass

In [2]:
# Get the current week number
week_number_email = int(input("Please, insert week number here: "))

In [3]:
email_sender = "ta.data.ironhack@gmail.com"
email_password = getpass.getpass("Your password (token provided by google for Python app):")

Your password (token provided by google for Python app):········


In [7]:
# create driver instance
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


In [4]:
# define path for secrets.txt

DATA_PATH =  pathlib.Path()

with open(DATA_PATH.joinpath('secrets.txt')) as f:
    secrets = f.read().splitlines() 

In [9]:
# open the website
driver.get('https://campus-tools.ironhack.com/#/cohorts/618138e279542a002c897068/show/course_progress/students')




In [10]:
# E-mail box:
email_box = driver.find_element(By.ID, "identifierId")

email_box.clear()

email_box.send_keys(secrets[0])



In [11]:
# Advance button:
advance_1 = driver.find_element(By.ID, "identifierNext")

advance_1.click()

In [12]:
# Password box:
pass_box = driver.find_element(By.NAME, "password")
time.sleep(0.5)
pass_box.clear()
time.sleep(1.5)
pass_box.send_keys(secrets[1])


In [13]:
# Advance button:
advance_2 = driver.find_element(By.ID, "passwordNext")
time.sleep(1.5)
advance_2.click()

In [14]:
# Get student portal link again because sometimes the previous page gets stuck
driver.get('https://campus-tools.ironhack.com/#/cohorts/618138e279542a002c897068/show/course_progress/students')


In [15]:
def get_macro_student_table(
    driver: webdriver) -> pd.DataFrame:

    #Find all rows of Weeks Separators and store in variable:

    students_table = (
        driver.find_element(
            By.CSS_SELECTOR,
            "div[class^='ra-field ra-field-undefined']")
    )

    students_table_body = (
        students_table.find_element(
            By.CSS_SELECTOR,
            "tbody[class^='MuiTableBody-root datagrid-body']")
    )

    html_table = students_table_body.get_attribute('outerHTML')

    soup = BeautifulSoup(html_table, 'html.parser')

    # images url
    images_url = soup.find_all('img', attrs = {'class':'MuiAvatar-img'})
    images_url = [image_link['src'] for image_link in images_url]

    # remaining student info
    students_info = soup.find_all("span", attrs = {'class':'MuiTypography-root MuiTypography-body2'})

    students_name = []
    unit_progress = []
    req_completed_labs = []
    for i in range(0, len(students_info), 5):
        group = students_info[i:i+5]
        students_name.append(group[0].text)
        unit_progress.append(group[2].text)
        req_completed_labs.append(group[3].text)

    labs_dataframe = pd.DataFrame(
            {'student_name': students_name,
            'required_labs_completion': req_completed_labs,
            'student_image_url': images_url
            }
    )

    labs_dataframe = (
        labs_dataframe.
        assign(completed_required_labs = labs_dataframe['required_labs_completion'].str.split('/').str[0]).
        assign(total_required_labs = labs_dataframe['required_labs_completion'].str.split('/').str[-1]).
        drop(columns = "required_labs_completion").
        astype({"completed_required_labs":int, "total_required_labs":int })
    )
    
    labs_dataframe["completion_pctg"] = (
        round(labs_dataframe['completed_required_labs'] / labs_dataframe['total_required_labs'], 2)
    )
    
    return labs_dataframe

    

In [16]:
macro_table = get_macro_student_table(driver)
students_list = macro_table["student_name"].unique()
students_list


array(['Adrian Flores', 'Aferdita Zherka Zherka', 'Alicia Andrés',
       'Amelie Haberland', 'Angela Arredondo Mendoza', 'Damiano Serra',
       'Daniel J Mendez Borges', 'Fabio Blank Da Costa',
       'Federico Cavanagh', 'Francisco Javier De Las Heras Jimenez',
       'Gregor Seegers', 'Joaquin Valentin', 'Manouk Meilof',
       'Marc Gavaldà', 'María Vázquez Casares', 'Max Grempel',
       'Shilei Tan Tan', 'Suparna Mandal', 'Vanessa Andrade',
       'Óscar Iglesias Roqueiro'], dtype=object)

### Iterate through each student tab:


In [17]:
# main window handle
wait = WebDriverWait(driver, 10)
original_window = driver.current_window_handle

assert len(driver.window_handles) == 1

# find students rows
students_rows = driver.find_elements(By.CSS_SELECTOR,"tr[class^='MuiTableRow-root jss']")
print(f'Found {len(students_rows)-1} students. Starting iteration through each of them.')


Found 20 students. Starting iteration through each of them.


In [18]:
# loop each student tab
all_students_table = pd.DataFrame()
for i in range(1, len(students_rows)):
    # open student page 
    students_rows[i].click()

    # wait until there's a new tab or window
    wait.until(EC.number_of_windows_to_be(2))

    #switch driver to new tab
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            break
    time.sleep(5)

    student_table = driver.find_element(By.CSS_SELECTOR, "div[class^='ra-field ra-field-course.deliveries']") 

    html_table = student_table.get_attribute('outerHTML')

    soup = BeautifulSoup(html_table, 'html.parser')

    # soup driver
    all_table_elements = soup.find_all("td")

    lab_name = []
    status = []
    submission_date = []

    for table_index in range(0, len(all_table_elements), 7):
        group = all_table_elements[table_index:table_index+7]
        lab_name.append(group[0].text)
        status.append(group[2].text)
        submission_date.append(group[4].text)
        
        
    students_df = pd.DataFrame(
        {'lab_name': lab_name,
        'lab_status': status,
        'submission_date': submission_date
        }
    )

    students_df["student_name"] = students_list[i-1]

    all_students_table = pd.concat([all_students_table, students_df])

    print(f'finished appending data for student {students_list[i-1]}.')

    # close the tab or window
    driver.close()

    # switch back to the old tab or window
    driver.switch_to.window(original_window)

    time.sleep(3)


            
        





finished appending data for student Adrian Flores.
finished appending data for student Aferdita Zherka Zherka.
finished appending data for student Alicia Andrés.
finished appending data for student Amelie Haberland.
finished appending data for student Angela Arredondo Mendoza.
finished appending data for student Damiano Serra.
finished appending data for student Daniel J Mendez Borges.
finished appending data for student Fabio Blank Da Costa.
finished appending data for student Federico Cavanagh.
finished appending data for student Francisco Javier De Las Heras Jimenez.
finished appending data for student Gregor Seegers.
finished appending data for student Joaquin Valentin.
finished appending data for student Manouk Meilof.
finished appending data for student Marc Gavaldà.
finished appending data for student María Vázquez Casares.
finished appending data for student Max Grempel.
finished appending data for student Shilei Tan Tan.
finished appending data for student Suparna Mandal.
fini

In [20]:
test_table = all_students_table.copy()

In [26]:
len(test_table["student_name"].unique())

20

In [22]:

all_students_table = (
    all_students_table.
    assign(lab = all_students_table["lab_name"].str.split("Required").str[0]).
    assign(required = (
        all_students_table["lab_name"].
        str.contains("Required").
        replace(True,"Required").
        replace(False, "Optional")
        )
    ).
    drop(columns="lab_name")
)

labs_cols = ['student_name', 'lab', 'required', 'lab_status', 'submission_date'] 
all_students_table = all_students_table[labs_cols]

all_students_table


,student_name,lab,required,lab_status,submission_date
0,Adrian Flores,Lab Git,Required,Delivered,07/09/2022
1,Adrian Flores,Lab Lists,Required,Delivered,09/09/2022
2,Adrian Flores,"Lab Dicts, sets, tuples",Required,Delivered,10/09/2022
3,Adrian Flores,Lab Strings,Required,Delivered,14/09/2022
4,Adrian Flores,Lab Intro to pandas,Required,Delivered,17/09/2022
...,...,...,...,...,...
37,Óscar Iglesias Roqueiro,Lab Imbalance,Required,Pending,
38,Óscar Iglesias Roqueiro,Lab Problems in ML,Required,Pending,
39,Óscar Iglesias Roqueiro,Lab Unsupervised Learning,Required,Pending,
40,Óscar Iglesias Roqueiro,Lab Deep Learning,Required,Pending,


In [ ]:
def clean_students_table(
    students_df : pd.DataFrame
    ) -> pd.DataFrame :

    students_df = (
        students_df.
        assign(lab = students_df["lab_name"].str.split("Required").str[0]).
        assign(required = (
            students_df["lab_name"].
            str.contains("Required").
            replace(True,"Required").
            replace(False, "Optional")
            )
        ).
        drop(columns="lab_name")
    )

    labs_cols = ['student_name', 'lab', 'required', 'lab_status', 'submission_date'] 
    students_df = students_df[students_df]

    return students_df


In [14]:
# Create a column with week number as numeric type
labs_overall["week_num"] = labs_overall["week"].apply(lambda x: int(x.split()[1]))

In [15]:
# Clean the lab_name column to remove "\nRequired"
labs_overall["lab_name"] = labs_overall["lab_name"].apply(lambda x: x.split("\n")[0])

In [17]:
# Get a list of students and their emails and create a dictionary with both lists
student_names = list(labs_overall["student_name"].unique())
student_names.remove("Manouk Meilof")

student_emails = ["adrianfg88@gmail.com",
"zherka.aferdita@outlook.com",
"aliciaandressalguero@gmail.com",
"amelie.haberland@ironhack.com",
"angela.arredondo99@gmail.com",
"damsvpn@protonmail.com",
"djmborges92@gmail.com",
"fabioblankc@gmail.com",
"fedecavanagh@live.com",
"javierherasjimenez@hotmail.com",
"gregor.seegers@gmail.com",
"quino117@hotmail.com",
"gavalda.marc@gmail.com",
"maria.vazquez.casares@gmail.com",
"maximilian.grempel@gmail.com",
"tanshilei@gmail.com",
"roque.ourense@gmail.com",
"suparna.sahoo@gmail.com",
"anvergara95@gmail.com"]

students_dict = dict(zip(student_names, student_emails))

In [18]:
students_dict

{'Adrian Flores': 'adrianfg88@gmail.com',
 'Aferdita Zherka Zherka': 'zherka.aferdita@outlook.com',
 'Alicia Andrés': 'aliciaandressalguero@gmail.com',
 'Amelie Haberland': 'amelie.haberland@ironhack.com',
 'Angela Arredondo Mendoza': 'angela.arredondo99@gmail.com',
 'Damiano Serra': 'damsvpn@protonmail.com',
 'Daniel J Mendez Borges': 'djmborges92@gmail.com',
 'Fabio Blank Da Costa': 'fabioblankc@gmail.com',
 'Federico Cavanagh': 'fedecavanagh@live.com',
 'Francisco Javier De Las Heras Jimenez': 'javierherasjimenez@hotmail.com',
 'Gregor Seegers': 'gregor.seegers@gmail.com',
 'Joaquin Valentin': 'quino117@hotmail.com',
 'Marc Gavaldà': 'gavalda.marc@gmail.com',
 'María Vázquez Casares': 'maria.vazquez.casares@gmail.com',
 'Max Grempel': 'maximilian.grempel@gmail.com',
 'Shilei Tan Tan': 'tanshilei@gmail.com',
 'Suparna Mandal': 'roque.ourense@gmail.com',
 'Óscar Iglesias Roqueiro': 'suparna.sahoo@gmail.com',
 'Vanessa Andrade': 'anvergara95@gmail.com'}

In [19]:
for student in student_names:
    
    # Filtering by student
    student_sub = labs_overall[labs_overall["student_name"] == student]
    
    # Filtering by week number
    student_sub_week = student_sub[student_sub["week_num"] <= week_number_email]
    
    # Count delivered and not delivered labs
    submissions = student_sub_week["lab_status"].value_counts()
    delivered = submissions["Delivered"]
    try:
        not_delivered = submissions[""]
    except:
        not_delivered = 0
    
    # Get the lab names of non delivered ones
    pending_labs = student_sub_week[student_sub_week["lab_status"] == ""]["lab_name"].values.tolist()
    pending_labs_strings = "\n".join(pending_labs)
    
    # Calculate the percentage of delivered labs
    lab_percent = round((delivered / (delivered+not_delivered))*100)
                        
    if lab_percent < 80:
        body = f'''Dear {student.split()[0]}, we have been checking your lab sumissions status. From a total of {delivered + not_delivered} labs, you have submitted {delivered}, this represents the {lab_percent}% of the total until now.
Remember that ironhack rules mark 80% as the threshold to complete the course. Don't hesitate in contacting us if you need help to meet that threshold. We will be happy to support you!

The labs that you are missing are:
{pending_labs_strings}
        
Ironhack Teaching Team
        '''
                        
    elif lab_percent == 100:
        body = f'''Dear {student.split()[0]}, we have been checking your lab sumissions status. From a total of {delivered + not_delivered} labs, you have submitted {delivered}, this represents the {lab_percent}% of the total until now.
You have submitted all the labs so far, so congratulations for this hard job!
        
Ironhack Teaching Team
        '''
    
    else:
        body = f'''Dear {student.split()[0]}, we have been checking your lab sumissions status. From a total of {delivered + not_delivered} labs, you have submitted {delivered}, this represents the {lab_percent}% of the total until now.
You are over the 80% threshold marked by Ironhack to complete the course so congratulations! You are properly following the course progress.
Anyway,there are some labs that are still missing:
{pending_labs_strings}

Keep going with the hard job!
        
Ironhack Teaching Team
        '''
        
    email_receiver = "miguelsimonmoya@gmail.com" #students_dict[student]
    subject = "Ironhack lab status"
    
    em = EmailMessage()
    em["From"] = email_sender
    em["To"] = email_receiver
    em["subject"] = subject
    em.set_content(body)

    context = ssl.create_default_context()

    with smtplib.SMTP_SSL("smtp.gmail.com", context = context) as smtp:
        smtp.login(email_sender, email_password)
        smtp.sendmail(email_sender, email_receiver, em.as_string())
    
    print(student)                    
    time.sleep(2)
                        
                        

Adrian Flores
Aferdita Zherka Zherka
Alicia Andrés
Amelie Haberland
Angela Arredondo Mendoza
Damiano Serra
Daniel J Mendez Borges
Fabio Blank Da Costa
Federico Cavanagh
Francisco Javier De Las Heras Jimenez
Gregor Seegers
Joaquin Valentin
Marc Gavaldà
María Vázquez Casares
Max Grempel
Shilei Tan Tan
Suparna Mandal
Óscar Iglesias Roqueiro
Vanessa Andrade


In [20]:
student_sub = labs_overall[labs_overall["student_name"] == "Angela Arredondo Mendoza"]
    
# Filtering by week number
student_sub_week = student_sub[student_sub["week_num"] <= week_number_email]

In [21]:
student_sub_week

,week,lab_name,student_name,lab_status,delivery_date,week_num
4,Week 1,Lab Strings,Angela Arredondo Mendoza,Delivered,14/9/2022,1
24,Week 1,"Lab Dicts, sets, tuples",Angela Arredondo Mendoza,Delivered,15/9/2022,1
44,Week 1,Lab Lists,Angela Arredondo Mendoza,Delivered,11/9/2022,1
64,Week 1,Lab Git,Angela Arredondo Mendoza,Delivered,8/9/2022,1
84,Week 2,Lab DataFrame Calculations,Angela Arredondo Mendoza,Delivered,24/9/2022,2
104,Week 2,Lab Data Cleaning,Angela Arredondo Mendoza,Delivered,22/9/2022,2
124,Week 2,Lab Import-Export,Angela Arredondo Mendoza,Delivered,18/9/2022,2
144,Week 2,Lab Intro to pandas,Angela Arredondo Mendoza,Delivered,18/9/2022,2
164,Week 3,Lab Descriptive Stats,Angela Arredondo Mendoza,Delivered,1/10/2022,3
184,Week 3,Lab Advanced Pandas,Angela Arredondo Mendoza,Delivered,30/9/2022,3


In [2]:
labs_table_df = pd.read_csv("labs_table.csv")

In [4]:
angela_df = labs_table_df[labs_table_df["student_name"].str.contains("Ang")]

In [5]:
angela_df

,week,lab_name,student_name,lab_status,delivery_date,week_num
4,Week 1,Lab Strings,Angela Arredondo Mendoza,Delivered,14/09/2022,1
24,Week 1,"Lab Dicts, sets, tuples",Angela Arredondo Mendoza,Delivered,15/09/2022,1
44,Week 1,Lab Lists,Angela Arredondo Mendoza,Delivered,11/09/2022,1
64,Week 1,Lab Git,Angela Arredondo Mendoza,Delivered,08/09/2022,1
84,Week 2,Lab DataFrame Calculations,Angela Arredondo Mendoza,Delivered,24/09/2022,2
104,Week 2,Lab Data Cleaning,Angela Arredondo Mendoza,Delivered,22/09/2022,2
124,Week 2,Lab Import-Export,Angela Arredondo Mendoza,Delivered,18/09/2022,2
144,Week 2,Lab Intro to pandas,Angela Arredondo Mendoza,Delivered,18/09/2022,2
164,Week 3,Lab Descriptive Stats,Angela Arredondo Mendoza,Delivered,01/10/2022,3
184,Week 3,Lab Advanced Pandas,Angela Arredondo Mendoza,Delivered,30/09/2022,3
